# Jon Conway: Game of Life
Notebook-based implementation using ipycanvas

In [ ]:
import copy
from ipycanvas import Canvas, hold_canvas
from time import sleep

### Define the initial states:

In [18]:
# instantiate variables
playground_size = (400, 600)
cell = {}
grid = {}
next_gen = {}
x_array = []
y_array = []
x = y = 0
col = row = 0
num_generations = 10

In [19]:
# build the initial empty grid:
for row in range(0,100):
    #print("row (y): ", row)
    for col in range(0,100):
        cell = {"x":x, "y":y, "size":9, "active":0, "row":row, "col":col}
        #print("row (y): ", row, "col (x): ", col, cell)
        grid[(row,col)]=(cell)
        x += 10
        col += 1

    x = 0
    col = 0
    y += 10

In [20]:
# build the initial population:
#grid[(1,1)] = {'x': 10, 'y': 10, 'size': 10, 'active': 1, 'row': 1, 'col': 1}
#grid[(0, 0)]={'x': 0, 'y': 0, 'size': 10, 'active': 0, 'row': 0, 'col': 0}

# define an oszillator:
#grid[(1, 1)]={'x': 10, 'y': 10, 'size': 9, 'active': 1, 'row': 1, 'col': 1}
#grid[(1, 2)]={'x': 20, 'y': 10, 'size': 9, 'active': 1, 'row': 1, 'col': 2}
#grid[(1, 3)]={'x': 30, 'y': 10, 'size': 9, 'active': 1, 'row': 1, 'col': 3}
grid[(1, 1)]['active']=1
grid[(1, 2)]['active']=1
grid[(1, 3)]['active']=1

# define the r-pentomino:
grid[(6, 5)]['active']=1
grid[(6, 6)]['active']=1
grid[(7, 4)]['active']=1
grid[(7, 5)]['active']=1
grid[(8, 5)]['active']=1

In [21]:
#grid
next_gen = copy.deepcopy(grid)

### Cells for testing:

In [ ]:
# create a copy of the current state of the grid in order not to overwrite the status when moving over the cells:

for cell in grid.keys():
    # get the x and y postion of the active cell:
    x_pos = cell[0]
    y_pos = cell[1]

    nw = grid.get((x_pos-1,y_pos-1),{}).get('active') or 0
    n = grid.get((x_pos,y_pos-1),{}).get('active') or 0
    ne = grid.get((x_pos+1,y_pos-1),{}).get('active') or 0
    w = grid.get((x_pos-1,y_pos),{}).get('active') or 0
    e = grid.get((x_pos+1,y_pos),{}).get('active') or 0
    sw = grid.get((x_pos-1,y_pos+1),{}).get('active') or 0
    s = grid.get((x_pos,y_pos+1),{}).get('active') or 0
    se = grid.get((x_pos+1,y_pos+1),{}).get('active') or 0

    res = nw + n + ne + w + e + sw + s + se
    #print("cell: ", x_pos, y_pos, "active: ", grid.get((x_pos,y_pos),{}).get('active'), "nw=", nw, "n=", n, "ne=", ne, "w=", w, "e=", e, "sw=", sw, "s=", s, "se=", se, " result: ", res)
        
    if grid.get((x_pos,y_pos),{}).get('active') == 1 and res < 2:
        next_gen[(x_pos,y_pos)]['active']=0
        #print("result for the cell at position {} = {} the cell dies by underpopulation {}".format(cell, res, next_gen[(x_pos,y_pos)]))
    elif grid.get((x_pos,y_pos),{}).get('active') == 1 and (res == 2 or res == 3):
        next_gen[(x_pos,y_pos)] = grid[cell]
        #print("result for the cell at position {} = {} the cell lives on {}".format(cell, res, next_gen[(x_pos,y_pos)]))
    elif grid.get((x_pos,y_pos),{}).get('active') == 1 and res > 3:
        next_gen[(x_pos,y_pos)]['active']=0
        #print("result for the cell at position {} = {} the cell dies by overpopulation {}".format(cell, res, next_gen[(x_pos,y_pos)]))
    elif grid.get((x_pos,y_pos),{}).get('active') == 0 and res == 3:
        next_gen[(x_pos,y_pos)]['active']=1
        #print("result for the cell at position {} = {} the cell becomes a live cell {}".format(cell, res, next_gen[(x_pos,y_pos)]))
    
    #print(x_pos, y_pos, res, grid[cell])

grid = copy.deepcopy(next_gen)


In [ ]:
#grid = copy.deepcopy(next_gen)

In [ ]:
#next_gen

In [ ]:
x_array = []
y_array = []

for cell in grid.keys():
    #print(cell, grid[cell])
    # get the x and y postion of the active cell:
    x_pos = cell[0]
    y_pos = cell[1]

    if grid.get((x_pos,y_pos),{}).get('active'):
        x_array.append(grid.get((x_pos,y_pos),{}).get('x'))
        y_array.append(grid.get((x_pos,y_pos),{}).get('y'))

In [ ]:
canvas = Canvas(width=100, height=200)

#canvas.fill_rect(25, 25, 10)
canvas.fill_rects(x_array, y_array, 9, height=None)

canvas

### Functions for implementation in a loop:

In [22]:
def next_population(grid):
    # create a copy of the current state of the grid in order not to overwrite the status when moving over the cells:

    for cell in grid.keys():
        # get the x and y postion of the active cell:
        x_pos = cell[0]
        y_pos = cell[1]
    
        nw = grid.get((x_pos-1,y_pos-1),{}).get('active') or 0
        n = grid.get((x_pos,y_pos-1),{}).get('active') or 0
        ne = grid.get((x_pos+1,y_pos-1),{}).get('active') or 0
        w = grid.get((x_pos-1,y_pos),{}).get('active') or 0
        e = grid.get((x_pos+1,y_pos),{}).get('active') or 0
        sw = grid.get((x_pos-1,y_pos+1),{}).get('active') or 0
        s = grid.get((x_pos,y_pos+1),{}).get('active') or 0
        se = grid.get((x_pos+1,y_pos+1),{}).get('active') or 0
    
        res = nw + n + ne + w + e + sw + s + se
        #print("cell: ", x_pos, y_pos, "active: ", grid.get((x_pos,y_pos),{}).get('active'), "nw=", nw, "n=", n, "ne=", ne, "w=", w, "e=", e, "sw=", sw, "s=", s, "se=", se, " result: ", res)
            
        if grid.get((x_pos,y_pos),{}).get('active') == 1 and res < 2:
            next_gen[(x_pos,y_pos)]['active']=0
            #print("result for the cell at position {} = {} the cell dies by underpopulation {}".format(cell, res, next_gen[(x_pos,y_pos)]))
        elif grid.get((x_pos,y_pos),{}).get('active') == 1 and (res == 2 or res == 3):
            next_gen[(x_pos,y_pos)] = grid[cell]
            #print("result for the cell at position {} = {} the cell lives on {}".format(cell, res, next_gen[(x_pos,y_pos)]))
        elif grid.get((x_pos,y_pos),{}).get('active') == 1 and res > 3:
            next_gen[(x_pos,y_pos)]['active']=0
            #print("result for the cell at position {} = {} the cell dies by overpopulation {}".format(cell, res, next_gen[(x_pos,y_pos)]))
        elif grid.get((x_pos,y_pos),{}).get('active') == 0 and res == 3:
            next_gen[(x_pos,y_pos)]['active']=1
            #print("result for the cell at position {} = {} the cell becomes a live cell {}".format(cell, res, next_gen[(x_pos,y_pos)]))
        
        #print(x_pos, y_pos, res, grid[cell])
            
    return next_gen
    #grid = copy.deepcopy(next_gen)

In [23]:
def get_active_cells(grid):
    x_array = []
    y_array = []

    for cell, cell_info in grid.items():
        x_pos, y_pos = cell
        if cell_info.get('active') == 1:
            x_array.append(cell_info.get('x'))
            y_array.append(cell_info.get('y'))

    return x_array, y_array

In [ ]:
#next_gen = next_population(grid)

In [ ]:
#grid = copy.deepcopy(next_gen)

### Loop to draw the Canvas:

In [24]:
canvas = Canvas(width=500, height=800)
display(canvas)

# Number of steps in your animation
steps_number = 100

for i in range(steps_number):
    with hold_canvas():
        # Clear the old animation step
        canvas.clear()

        # calculate the next population: 
        next_gen = next_population(grid)

        # get the x and y arrays: 
        arrays = get_active_cells(grid)

        # Perfom all your drawings here
        canvas.fill_rects(arrays[0], arrays[1], 9, height=None)

        # copy the result of the next generation to the 
        grid = copy.deepcopy(next_gen)
        
    # Animation frequency ~50Hz = 1./50. seconds
    sleep(0.02)

Canvas(height=800, width=500)